# Phase-2 Handling Missing Values

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import warnings;
warnings.filterwarnings("ignore")
import re
#pd.set_option("display.max_columns",None)
#pd.set_option("display.max_rows",None)

In [2]:
df=pd.read_csv("./data/scout_cleaned_2.csv")

### 1) Body Type

In [3]:
df["body_type"].value_counts(dropna=False)

body_type
Sedans           7902
Station wagon    3552
Compact          3152
Van               783
Other             289
Transporter        88
NaN                60
Off-Road           56
Coupe              25
Convertible         8
Name: count, dtype: int64

In [4]:
df[df["body_type"] == np.nan]

,make_model,body_type,price,vat,km,prev_owner,hp,inspection_new,warranty,body_color,...,gears,electricity_consumption,year,new_used,fuel_type,next_inspection_date,upholstery_material,consumption_comb,consumption_city,consumption_country


In [5]:
df.groupby('make_model')['body_type'].transform(lambda x: x.mode()[0])

0        Sedans
1        Sedans
2        Sedans
3        Sedans
4        Sedans
          ...  
15910       Van
15911       Van
15912       Van
15913       Van
15914       Van
Name: body_type, Length: 15915, dtype: object

In [6]:
df["body_type"].fillna(df.groupby('make_model')['body_type'].transform(lambda x: x.mode()[0]), inplace=True)

### 2) Registration

In [7]:
df.loc[((df['new_used' # was 'Type'
         ] == 'New') & df.year.isnull() # was df.registration.isnull()
       ), 'year' # was 'registration'
] 

122     NaN
710     NaN
734     NaN
741     NaN
743     NaN
         ..
15888   NaN
15898   NaN
15903   NaN
15908   NaN
15910   NaN
Name: year, Length: 1547, dtype: float64

In [8]:
# df.loc[((df['Type'] == 'New') & df.registration.isnull()), 'registration'] = 2019 
df.loc[((df['new_used'] == 'New') & df.year.isnull()), 'year'] = 2019 

In [9]:
# df.groupby('registration').km.mean()
df.groupby('year').km.mean()

year
2016.0    77442.520958
2017.0    41754.940709
2018.0    18042.018295
2019.0     1654.068774
Name: km, dtype: float64

* Since the registration of all new cars is 2019, null values of year column of new cars filled with 2019.

In [10]:
# df.loc[(df['registration'].isnull()) & (df.km <= 5000),'registration'] =2019
# df.loc[(df['registration'].isnull()) & (df.km > 50000),'registration'] =2016
# df.loc[(df['registration'].isnull()) & (df.km < 19000),'registration'] =2018
# df.loc[(df['registration'].isnull()) & (df.km < 40000),'registration'] =2017

df.loc[(df['year'].isnull()) & (df.km <= 5000),'year'] =2019
df.loc[(df['year'].isnull()) & (df.km > 50000),'year'] =2016
df.loc[(df['year'].isnull()) & (df.km < 19000),'year'] =2018
df.loc[(df['year'].isnull()) & (df.km < 40000),'year'] =2017

* Rest of the null values were filled according to km of autos.

In [11]:
# df[(df['make_model'] == 'Audi A3') & (df['body_type'] == 'Sedans')].groupby('registration').price.mean()
df[(df['make_model'] == 'Audi A3') & (df['body_type'] == 'Sedans')].groupby('year').price.mean()

year
2016.0    16702.052388
2017.0    19970.030631
2018.0    22544.715092
2019.0    24859.973913
Name: price, dtype: float64

In [12]:
# df[(df['make_model'] == 'Opel Insignia')].groupby('registration').price.mean()
df[(df['make_model'] == 'Opel Insignia')].groupby('year').price.mean()

year
2016.0    13606.709507
2017.0    16995.140917
2018.0    21390.167750
2019.0    32103.197015
Name: price, dtype: float64

In [13]:
# df.loc[(df['registration'].isnull())]
df.loc[(df['year'].isnull())]

,make_model,body_type,price,vat,km,prev_owner,hp,inspection_new,warranty,body_color,...,gears,electricity_consumption,year,new_used,fuel_type,next_inspection_date,upholstery_material,consumption_comb,consumption_city,consumption_country
5237,Audi A3,Sedans,25400,NaN,NaN,NaN,85.0,NaN,NaN,Grey,...,7.0,0,NaN,NaN,NaN,NaN,Cloth,3.9,4.1,3.7
5329,Audi A3,Sedans,24900,NaN,NaN,NaN,85.0,NaN,24.0,White,...,7.0,0,NaN,Pre-registered,diesel,NaN,Cloth,3.9,4.1,3.7
12546,Opel Insignia,Sedans,33800,NaN,NaN,NaN,100.0,NaN,24.0,White,...,6.0,0,NaN,Employee's car,diesel,NaN,Part leather,5.4,7.0,4.6
12878,Opel Insignia,Station wagon,31318,VAT deductible,NaN,NaN,100.0,NaN,24.0,Grey,...,6.0,0,NaN,Used,diesel,NaN,Full leather,4.3,NaN,NaN


In [14]:
# df['registration']=df['registration'].fillna(2019)
df['year']=df['year'].fillna(2019)

In [15]:
# df['registration'] = 2019 - df['registration']
df['year'] = 2019 - df['year']

# df = df.rename(columns = {'registration': 'age'})
df = df.rename(columns = {'year': 'age'})

* year column converted to age column

### 3) VAT

In [16]:
df.vat.value_counts(dropna=False)

vat
VAT deductible      10980
NaN                  4509
Price negotiable      426
Name: count, dtype: int64

In [17]:
df['vat'] = df['vat'].fillna('VAT undeductible') 

* All null values of vat column assigned as 'VAT undeductible'

### 4) KM

In [18]:
median_group_km = df.groupby(['make_model','body_type','age']).km.transform(lambda x: x.median())

In [19]:
df["km"].fillna(median_group_km, inplace=True)

In [20]:
df["km"].fillna(df.groupby(['make_model','age']).km.transform(lambda x: x.median()), inplace=True)

* km column filled with groups age, make_model and body_type

### 5) Previous Owner

In [21]:
index=df[df["km"]<10].index

In [22]:
df.loc[index,"prev_owner"]=0.0

In [23]:
# df.loc[(df["Type"]=="New") & (df["prev_owner"].isnull()),"prev_owner"]=1.0
df.loc[(df["new_used"]=="New") & (df["prev_owner"].isnull()),"prev_owner"]=1.0

In [24]:
df.loc[((df["age"]==2019) | (df["km"]<5000)) & (df["prev_owner"].isnull()),"prev_owner"]=1.0

In [25]:
mode_group_po = df.groupby(['age'])['prev_owner'].transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [26]:
df["prev_owner"].fillna(mode_group_po, inplace=True)

### hp 

In [27]:
# mode_hp = df[~(df['Displacement'].isnull())]\
#               .groupby(['make_model','body_type','Displacement'])['hp']\
#               .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

mode_hp = df[~(df['displacement'].isnull())]\
              .groupby(['make_model','body_type','displacement'])['hp']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [28]:
# df.loc[~(df['Displacement'].isnull()),"hp"] = df.loc[~(df['Displacement'].isnull()),"hp"].fillna(mode_hp)
df.loc[~(df['displacement'].isnull()),"hp"] = df.loc[~(df['displacement'].isnull()),"hp"].fillna(mode_hp)

In [29]:
# mode_hp1 = df[~(df['Cylinders'].isnull())]\
#               .groupby(['make_model','body_type','Cylinders'])['hp']\
#               .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

mode_hp1 = df[~(df['cylinders'].isnull())]\
              .groupby(['make_model','body_type','cylinders'])['hp']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [30]:
# df.loc[~(df['Cylinders'].isnull()),"hp"] = df.loc[~(df['Cylinders'].isnull()),"hp"].fillna(mode_hp1)

df.loc[~(df['cylinders'].isnull()),"hp"] = df.loc[~(df['cylinders'].isnull()),"hp"].fillna(mode_hp1)

In [31]:
mode_hp2 = df.groupby(['make_model','body_type'])['hp']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [32]:
df['hp'] = df['hp'].fillna(mode_hp2)

 ### 7) Type

In [33]:
# df['Type'] = df['Type'].fillna('Used')
df['new_used'] = df['new_used'].fillna('Used')

### 8) Next Inspection

In [34]:
# df.drop(["Next Inspection"],axis=1,inplace=True)
df.drop(["next_inspection_date"],axis=1,inplace=True)

### 9) Inspection New

In [35]:
# df['Inspection new'].value_counts(dropna=False)
df['inspection_new'].value_counts(dropna=False)

inspection_new
NaN    11983
Yes     3932
Name: count, dtype: int64

In [36]:
# df['Inspection new'] = df['Inspection new'].fillna('No')
df['inspection_new'] = df['inspection_new'].fillna('No')

### 10) Warranty(months)

In [37]:
# df['Warranty(months)'] = df['Warranty(months)'].fillna(0.0)
df['warranty'] = df['warranty'].fillna(0.0)

* warranty column was filled with 0's

### 11) Body Color

In [38]:
# mode_color = df.groupby(['make_model','body_type'])['Body Color']\
#               .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

mode_color = df.groupby(['make_model','body_type'])['body_color']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [39]:
# df['Body Color'] = df['Body Color'].fillna(mode_color)

df['body_color'] = df['body_color'].fillna(mode_color)

In [40]:
# mode_color1 = df.groupby(['make_model'])['Body Color']\
#               .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

mode_color1 = df.groupby(['make_model'])['body_color']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [41]:
# df['Body Color'] = df['Body Color'].fillna(mode_color1)
df['body_color'] = df['body_color'].fillna(mode_color1)

* body_color column was filled according to groups make_model, body_type

### 11) Paint Type

In [42]:
# mode_paint = df.groupby(['make_model','body_type'])['Paint Type']\
#               .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)
mode_paint = df.groupby(['make_model','body_type'])['paint_type']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [43]:
# df['Paint Type'] = df['Paint Type'].fillna(mode_paint)
df['paint_type'] = df['paint_type'].fillna(mode_paint)

In [44]:
# mode_paint1 = df.groupby(['make_model'])['Paint Type']\
#               .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

mode_paint1 = df.groupby(['make_model'])['paint_type']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [45]:
# df['Paint Type'] = df['Paint Type'].fillna(mode_paint1)
df['paint_type'] = df['paint_type'].fillna(mode_paint1)

* paint_type column was filled according to groups make_model, body_type

### 12) Body Color Original

In [46]:
# df.drop(["Body Color Original"],axis=1,inplace=True)

### 13) Nr. of Doors

In [47]:
# mode_nr_doors = df.groupby(['make_model','body_type'])['Nr. of Doors']\
#               .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)
mode_nr_doors = df.groupby(['make_model','body_type'])['nr_of_doors']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [48]:
# df['Nr. of Doors'] = df['Nr. of Doors'].fillna(mode_nr_doors)
df['nr_of_doors'] = df['nr_of_doors'].fillna(mode_nr_doors)

In [49]:
# mode_nr_doors1 = df.groupby(['make_model'])['Nr. of Doors']\
#               .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

mode_nr_doors1 = df.groupby(['make_model'])['nr_of_doors']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [50]:
# df['Nr. of Doors'] = df['Nr. of Doors'].fillna(mode_nr_doors1)
df['nr_of_doors'] = df['nr_of_doors'].fillna(mode_nr_doors1)

* nr. of seats column was filled according to groups make_model, body_type

### 14) Nr. of Seats

In [51]:
# mode_nr_seats = df.groupby(['make_model','body_type'])['Nr. of Seats']\
#               .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)
mode_nr_seats = df.groupby(['make_model','body_type'])['nr_of_seats']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [52]:
# df['Nr. of Seats'] = df['Nr. of Seats'].fillna(mode_nr_seats)
df['nr_of_seats'] = df['nr_of_seats'].fillna(mode_nr_seats)

In [53]:
# mode_nr_seats1 = df.groupby(['make_model'])['Nr. of Seats']\
#               .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)
mode_nr_seats1 = df.groupby(['make_model'])['nr_of_seats']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [54]:
# df['Nr. of Seats'] = df['Nr. of Seats'].fillna(mode_nr_seats1)
df['nr_of_seats'] = df['nr_of_seats'].fillna(mode_nr_seats1)

* nr_of_seats column was filled according to groups make_model, body_type

### Model Code

In [55]:
# df.drop(['Model Code'],axis=1,inplace=True)

### 15) Gearing Type

In [56]:
# df['Gearing Type'].value_counts(dropna=False)
df['gearing_type'].value_counts(dropna=False)

gearing_type
Manual            8149
Automatic         7297
Semi-automatic     469
Name: count, dtype: int64

### 16) Displacement

In [57]:
# mode_displacement = df.groupby(['make_model','body_type','hp'])['Displacement']\
#               .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

mode_displacement = df.groupby(['make_model','body_type','hp'])['displacement']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [58]:
# df['Displacement'] = df['Displacement'].fillna(mode_displacement)

df['displacement'] = df['displacement'].fillna(mode_displacement)

In [59]:
# mode_displacement1 = df.groupby(['make_model','body_type'])['Displacement']\
#               .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

mode_displacement1 = df.groupby(['make_model','body_type'])['displacement']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [60]:
# df['Displacement'] = df['Displacement'].fillna(mode_displacement1)

df['displacement'] = df['displacement'].fillna(mode_displacement1)

In [61]:
# mode_displacement2 = df.groupby(['make_model'])['Displacement']\
#               .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

mode_displacement2 = df.groupby(['make_model'])['displacement']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [62]:
# df['Displacement'] = df['Displacement'].fillna(mode_displacement2)

df['displacement'] = df['displacement'].fillna(mode_displacement2)

* Displacement column was filled according to groups hp, make_model, body_type

### 16) Weight

In [63]:
# mode_weight = df.groupby(['make_model', 'Displacement'])['Weight']\
#               .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

mode_weight = df.groupby(['make_model', 'displacement'])['weight']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [64]:
# df['Weight'] = df['Weight'].fillna(mode_weight)

df['weight'] = df['weight'].fillna(mode_weight)

In [65]:
# mode_weight1 = df.groupby(['make_model'])['Weight']\
#               .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

mode_weight1 = df.groupby(['make_model'])['weight']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [66]:
# df['Weight'] = df['Weight'].fillna(mode_weight1)

df['weight'] = df['weight'].fillna(mode_weight1)

In [67]:
# mode_weight2 = df.groupby(['make_model', 'hp'])['Weight']\
#               .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

mode_weight2 = df.groupby(['make_model', 'hp'])['weight']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [68]:
# df['Weight'] = df['Weight'].fillna(mode_weight2)

df['weight'] = df['weight'].fillna(mode_weight2)

In [69]:
# mode_weight3 = df.groupby(['body_type'])['Weight']\
#               .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

mode_weight3 = df.groupby(['body_type'])['weight']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [70]:
# df['Weight'] = df['Weight'].fillna(mode_weight3)

df['Weight'] = df['weight'].fillna(mode_weight3)

* weight column was filled according to groups hp, make_model, body_type

### 17) Cylinders

In [71]:
# mode_cylinders = df.groupby(['make_model', 'Displacement'])['Cylinders']\
#               .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

mode_cylinders = df.groupby(['make_model', 'displacement'])['cylinders']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [72]:
# df['Cylinders'] = df['Cylinders'].fillna(mode_cylinders)

df['cylinders'] = df['cylinders'].fillna(mode_cylinders)

In [73]:
# mode_cylinders1 = df.groupby(['make_model', 'Weight'])['Cylinders']\
#               .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

mode_cylinders1 = df.groupby(['make_model', 'weight'])['cylinders']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [74]:
# df['Cylinders'] = df['Cylinders'].fillna(mode_cylinders1)

df['cylinders'] = df['cylinders'].fillna(mode_cylinders1)

In [75]:
# mode_cylinders2 = df.groupby(['make_model'])['Cylinders']\
#               .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

mode_cylinders2 = df.groupby(['make_model'])['cylinders']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [76]:
# df['Cylinders'] = df['Cylinders'].fillna(mode_cylinders2)

df['cylinders'] = df['cylinders'].fillna(mode_cylinders2)

In [77]:
# df['Cylinders'] = df['Cylinders'].fillna(4.0)

df['cylinders'] = df['cylinders'].fillna(4.0)

* cylinders column was filled according to groups displacement, weight, make_mode

### 18) Drive Chain

In [78]:
# mode_drive_chain = df.groupby(['make_model', 'body_type'])['Drive chain']\
#               .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

mode_drive_chain = df.groupby(['make_model', 'body_type'])['drive_chain']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [79]:
# df['Drive chain'] = df['Drive chain'].fillna(mode_drive_chain)

df['drive_chain'] = df['drive_chain'].fillna(mode_drive_chain)

In [80]:
# mode_drive_chain1 = df.groupby(['make_model'])['Drive chain']\
#               .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

mode_drive_chain1 = df.groupby(['make_model'])['drive_chain']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [81]:
# df['Drive chain'] = df['Drive chain'].fillna(mode_drive_chain1)

df['drive_chain'] = df['drive_chain'].fillna(mode_drive_chain1)

* drive_chain column was filled according to groups make_model and body type

### 19) Fuel

In [82]:
# df['Fuel'].value_counts(dropna=False)
df['fuel_type'].value_counts(dropna=False)

fuel_type
benzine    8546
diesel     7296
gas          64
others        7
NaN           2
Name: count, dtype: int64

In [83]:
# not sure what to do about the NaN?

### 20) CO2 Emission(g CO2/km)

In [84]:
# mode_CO2 = df.groupby(['make_model','Displacement'])['CO2 Emission(g CO2/km)']\
#               .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

mode_CO2 = df.groupby(['make_model','displacement'])['co2_emission']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [85]:
# df['CO2 Emission(g CO2/km)'] = df['CO2 Emission(g CO2/km)'].fillna(mode_CO2)

df['co2_emission'] = df['co2_emission'].fillna(mode_CO2)

In [86]:
# mode_CO2_1 = df.groupby(['make_model'])['CO2 Emission(g CO2/km)']\
#               .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

mode_CO2_1 = df.groupby(['make_model'])['co2_emission']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [87]:
# df['CO2 Emission(g CO2/km)'] = df['CO2 Emission(g CO2/km)'].fillna(mode_CO2_1)

df['co2_emission'] = df['co2_emission'].fillna(mode_CO2_1)

In [88]:
# mode_CO2_2 = df.groupby(['body_type'])['CO2 Emission(g CO2/km)']\
#               .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

mode_CO2_2 = df.groupby(['body_type'])['co2_emission']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [89]:
# df['CO2 Emission(g CO2/km)'] = df['CO2 Emission(g CO2/km)'].fillna(mode_CO2_2)

df['co2_emission'] = df['co2_emission'].fillna(mode_CO2_2)

* 'CO2 Emission(g CO2/km)' column was filled according to groups make_model,displacement and body type

### 21) Emission Class

In [90]:
# mode_Emision_C= df.groupby(['CO2 Emission(g CO2/km)'])['Emission Class']\
#               .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

mode_Emision_C= df.groupby(['co2_emission'])['emission_class']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [91]:
# df['Emission Class'] = df['Emission Class'].fillna(mode_Emision_C)

df['emission_class'] = df['emission_class'].fillna(mode_Emision_C)

In [92]:
# mode_Emision_C1= df.groupby(['age'])['Emission Class']\
#               .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

mode_Emision_C1= df.groupby(['age'])['emission_class']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [93]:
# df['Emission Class'] = df['Emission Class'].fillna(mode_Emision_C1)

df['emission_class'] = df['emission_class'].fillna(mode_Emision_C1)

* emission class column was filled according to groups co2 emission and age

### 22) Gears

In [94]:
# mode_Gears= df.groupby(['make_model','Gearing Type'])['Gears']\
#               .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

mode_Gears= df.groupby(['make_model','gearing_type'])['gears']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [95]:
# df['Gears'] = df['Gears'].fillna(mode_Gears)

df['gears'] = df['gears'].fillna(mode_Gears)

In [96]:
# mode_Gears1= df.groupby(['Gearing Type'])['Gears']\
#               .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

mode_Gears1= df.groupby(['gearing_type'])['gears']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [97]:
# df['Gears'] = df['Gears'].fillna(mode_Gears1)

df['gears'] = df['gears'].fillna(mode_Gears1)

* gears column was filled according to groups make model and gear type

### 23) Country version

In [98]:
# df.drop(['Country version'], axis=1, inplace=True)

### 24) Upholstery Material

In [99]:
df['upholstery_material'].value_counts(dropna=False)

upholstery_material
Cloth           8421
NaN             4501
Part leather    1499
Full leather    1009
Other            368
Velour            60
alcantara         57
Name: count, dtype: int64

### 25) Upholstrey Color

In [100]:
# df.drop(['upholstery_color'], axis=1, inplace=True)

### 26) Consumption Combined

In [101]:
# coms = ((df['Consumption_city']+df['Consumption_country'])/2)

coms = ((df['consumption_city']+df['consumption_country'])/2)

In [102]:
# df['Consumption_combined'] = df['Consumption_combined'].fillna(coms)

df['consumption_comb'] = df['consumption_comb'].fillna(coms)

In [103]:
# mode_Cons= df.groupby(['CO2 Emission(g CO2/km)'])['Consumption_combined']\
#               .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

mode_Cons= df.groupby(['co2_emission'])['consumption_comb']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [104]:
# df['Consumption_combined'] = df['Consumption_combined'].fillna(mode_Cons)

df['consumption_comb'] = df['consumption_comb'].fillna(mode_Cons)

In [105]:
# mode_Cons1= df.groupby(['Displacement'])['Consumption_combined']\
#               .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

mode_Cons1= df.groupby(['displacement'])['consumption_comb']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [106]:
# df['Consumption_combined'] = df['Consumption_combined'].fillna(mode_Cons1)

df['consumption_comb'] = df['consumption_comb'].fillna(mode_Cons1)

In [107]:
# mode_Cons2= df.groupby(['make_model'])['Consumption_combined']\
#               .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

mode_Cons2= df.groupby(['make_model'])['consumption_comb']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [108]:
# df['Consumption_combined'] = df['Consumption_combined'].fillna(mode_Cons2)

df['consumption_comb'] = df['consumption_comb'].fillna(mode_Cons2)

* comsumption column first was filled with the mean of 'consumption_city','consumption_country' columns. Then was filled according to the column make_model.

## Filled data was saved to a csv file called "missings_filled_step2.csv"

In [109]:
df.to_csv("./data/scout_missings_filled_2.csv", index=False)